In [1]:
import tensorflow as tf

In [2]:
tf.reset_default_graph()
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
z = tf.placeholder(dtype=tf.int32, name="z")
f = x * y * z + 3
g = f + 4

In [28]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    f_val, g_val = sess.run([f, g], feed_dict={z: 4})
    
print(f_val, g_val)

51 55


In [71]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

tf.reset_default_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

batch_size = 1024
num_batches = int(np.ceil(m/batch_size))
n_epochs = 1000
learning_rate = 0.01
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
scaled_housing_data_plus_bias = (housing_data_plus_bias - housing_data_plus_bias.mean()) / np.std(housing_data_plus_bias)
X = tf.placeholder(dtype=tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse, var_list=[theta])
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

saver = tf.train.Saver()

def fetch_batch(epoch, X_data, y_data, batch_index, batch_size):
    start_idx = batch_index * batch_size
    end_idx = (batch_index + 1) * batch_size
    X_batch = X_data[start_idx:end_idx]
    y_batch = y_data[start_idx:end_idx]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval(feed_dict={X: scaled_housing_data_plus_bias, y: housing.target.reshape(-1,1)}))
            saver.save(sess, 'model_checkpoints/model', global_step = epoch)
        for batch_index in range(num_batches):
            X_batch, y_batch = fetch_batch(epoch, scaled_housing_data_plus_bias, housing.target.reshape(-1, 1),
                                          batch_index, batch_size)
            
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * num_batches + batch_index
                file_writer.add_summary(summary_str, step)
                
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = sess.run(theta)
file_writer.close()



Epoch 0 MSE = 10.662262
Epoch 100 MSE = 1.339042
Epoch 200 MSE = 1.3379322
Epoch 300 MSE = 1.3368399
Epoch 400 MSE = 1.3357655
Epoch 500 MSE = 1.3347068
Epoch 600 MSE = 1.3336638
Epoch 700 MSE = 1.3326361
Epoch 800 MSE = 1.3316244
Epoch 900 MSE = 1.3306257


In [44]:
data = np.array([[1,2,3],[4,5,6]])
data

array([[1, 2, 3],
       [4, 5, 6]])

array([[1., 2., 3., 1., 1., 1., 1.],
       [4., 5., 6., 1., 1., 1., 1.]])

In [69]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
training_op = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(mse)
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)

    best_theta = theta.eval()


Epoch 0 MSE = 10.2133255
Epoch 100 MSE = 2.0669382
Epoch 200 MSE = 1.5876637
Epoch 300 MSE = 1.4228784
Epoch 400 MSE = 1.366201
Epoch 500 MSE = 1.3466877
Epoch 600 MSE = 1.3399495
Epoch 700 MSE = 1.3376017
Epoch 800 MSE = 1.3367647
Epoch 900 MSE = 1.3364468
